In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import pandas as pd
from tqdm.auto import tqdm


In [34]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if DEVICE.type == "cuda":
    # Print CUDA details
    print(f"CUDA Device: {torch.cuda.get_device_name()}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated()/1024**2:.2f}MB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved()/1024**2:.2f}MB")

In [ ]:
DEVICE

In [36]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="sequential",
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.to(DEVICE)
    return model, tokenizer

model, tokenizer = load_model("databricks/dbrx-instruct") # or base

In [ ]:
model.eval()

In [5]:
def get_moe_metadata(model, input_ids):
    """
    Get both router logits and expert indices for all MoE layers in DBRX.
    
    Args:
        model: DBRX model
        input_ids: Input token IDs
        
    Returns:
        moe_metadata: Dictionary containing router logits and expert indices
    """
    router_logits_list = []
    expert_indices_list = []
    
    def hook_fn(module, input, output):
        # For DBRX, we need to capture router logits and expert indices
        # output typically contains: (selected_experts, route_prob, dispatch_mask, combine_weights, expert_layer_output)
        hidden_states = input[0]
        
        # Extract router logits - adjust based on actual DBRX implementation
        # This is a simplification and may need to be adjusted based on actual DBRX gate structure
        if hasattr(module, 'gate_proj'):
            logits = module.gate_proj(hidden_states)
            router_logits_list.append(logits.detach())
        
        # Store expert indices (typically first element of output tuple)
        if isinstance(output, tuple) and len(output) > 0:
            expert_indices_list.append(output[0].detach())
        
        return output
    
    hooks = []
    # Iterate through model layers to find MoE layers
    for name, module in model.named_modules():
        # Look for MoE layers - adapt based on DBRX architecture
        if 'moe' in name.lower() and 'block' in name.lower() and hasattr(module, 'gate'):
            hook = module.gate.register_forward_hook(hook_fn)
            hooks.append(hook)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
    for hook in hooks:
        hook.remove()

    moe_metadata = {
        'router_logits': torch.stack(router_logits_list) if router_logits_list else None,
        'expert_indices': torch.stack(expert_indices_list) if expert_indices_list else None
    }
    
    if moe_metadata['router_logits'] is not None:
        print(f"Router logits shape: {moe_metadata['router_logits'].shape}")
    if moe_metadata['expert_indices'] is not None:
        print(f"Expert indices shape: {moe_metadata['expert_indices'].shape}")
    
    return moe_metadata

def prepare_prompt(prompt, tokenizer, max_tokens=2048):
    """
    Prepare a prompt for processing, splitting if necessary to fit within model context.
    
    Args:
        prompt: The text prompt to prepare
        tokenizer: The model's tokenizer
        max_tokens: Maximum number of tokens per chunk (default: 2048)
        
    Returns:
        List of prompts that fit within token limit
    """
    # Check if the input is a list of lines/prompts
    if isinstance(prompt, list):
        all_prepared_prompts = []
        for single_prompt in prompt:
            # Process each line/prompt individually
            prepared_chunks = prepare_prompt(single_prompt, tokenizer, max_tokens)
            all_prepared_prompts.extend(prepared_chunks)
        return all_prepared_prompts
    
    # Process a single prompt
    tokens = tokenizer.encode(prompt)
    
    # If prompt is small enough, return as is
    if len(tokens) <= max_tokens:
        return [prompt]
    
    # Split into manageable chunks
    prepared_prompts = []
    
    # Decode tokens into chunks
    start_idx = 0
    while start_idx < len(tokens):
        end_idx = min(start_idx + max_tokens, len(tokens))
        chunk_tokens = tokens[start_idx:end_idx]
        chunk_text = tokenizer.decode(chunk_tokens)
        prepared_prompts.append(chunk_text)
        start_idx = end_idx
    
    print(f"Long prompt detected! Split into {len(prepared_prompts)} chunks.")
    return prepared_prompts

def process_text_file_for_expert_counts(file_path, model, tokenizer, output_path=None, max_tokens=4096):
    """
    Process a text file to analyze MoE routing and count tokens per expert in each layer.
    Saves a PyTorch file with expert token counts. Adapted for DBRX architecture.
    
    Args:
        file_path: Path to text file with prompts (one per line)
        model: DBRX model
        tokenizer: DBRX tokenizer
        output_path: Path to save PyTorch results (default: based on input filename)
        max_tokens: Maximum tokens per prompt chunk
    """
    # Load the text file
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Check if this is a GitHub code file
    if 'github.txt' in file_path:
        import re
        # Find all code blocks using the file pattern
        file_pattern = re.compile(r'.*\\b\\w+\\.(js|py|c|cpp|java|ts|rb|go|rs|cs|swift|kt|php)$', re.MULTILINE)
        
        # Find all matches (file headers)
        matches = list(file_pattern.finditer(content))
        
        # Extract code blocks between file headers
        raw_prompts = []
        for i in range(len(matches)):
            start_pos = matches[i].start()
            # If this is the last match, go to the end of the file
            if i == len(matches) - 1:
                end_pos = len(content)
            else:
                end_pos = matches[i+1].start()
            
            # Extract the code block including the file header
            code_block = content[start_pos:end_pos].strip()
            raw_prompts.append(code_block)
    else:
        # Regular text file processing (one prompt per line)
        raw_prompts = [line.strip() for line in content.split('\n') if line.strip()]
    
    print(f"Loaded {len(raw_prompts)} raw prompts from {file_path}")
    
    # Prepare prompts (handle large prompts by splitting)
    prompts = []
    for raw_prompt in raw_prompts:
        prepared_chunks = prepare_prompt(raw_prompt, tokenizer, max_tokens)
        prompts.extend(prepared_chunks)
    
    print(f"Processing {len(prompts)} prepared prompts (after splitting large ones)")
    
    # Set default output path if not provided
    if output_path is None:
        output_path = file_path.replace('.txt', '_expert_data.pt')
    
    # Initialize counter for expert usage
    # Structure: {layer_num: {expert_id: count}}
    expert_counts = {}
    
    # Calculate total tokens for progress bar
    total_tokens = 0
    for prompt in prompts:
        tokens = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)
        total_tokens += tokens.size(1)
    
    print(f"Total tokens to process: {total_tokens}")
    
    # Initialize progress bar
    pbar = tqdm(total=total_tokens, desc="Processing tokens")
    processed_tokens = 0
    
    # Process each prompt
    for prompt in prompts:
        # Tokenize the prompt
        tokens = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)
        seq_len = tokens.size(1)
        
        # Get MoE routing metadata
        moe_metadata = get_moe_metadata(model, tokens)
        
        if moe_metadata['expert_indices'] is None:
            print("No MoE layers detected or no routing information available")
            processed_tokens += seq_len
            pbar.update(seq_len)
            continue
        
        # Extract expert indices
        expert_indices = moe_metadata['expert_indices']  # shape: [num_layers, seq_len, top_k]
        num_moe_layers = expert_indices.size(0)
        
        # Initialize counter for this batch if needed
        for layer_idx in range(num_moe_layers):
            layer_num = layer_idx + 1  # 1-based layer indexing
            if layer_num not in expert_counts:
                expert_counts[layer_num] = {}
        
        # Count token routing for each layer
        for layer_idx in range(num_moe_layers):
            layer_num = layer_idx + 1  # 1-based layer indexing
            
            # Process each token in sequence
            for token_idx in range(seq_len):
                # Get experts selected for this token in this layer
                selected_experts = expert_indices[layer_idx, token_idx].cpu().numpy().tolist()
                
                # Count each expert
                for expert_id in selected_experts:
                    if expert_id not in expert_counts[layer_num]:
                        expert_counts[layer_num][expert_id] = 0
                    expert_counts[layer_num][expert_id] += 1
        
        # Update progress bar
        processed_tokens += seq_len
        pbar.update(seq_len)
    
    # Close progress bar
    pbar.close()
    
    # Convert counts to a simple tensor format for saving
    expert_token_counts = {}
    for layer_num in sorted(expert_counts.keys()):
        layer_data = expert_counts[layer_num]
        # Create a tensor with counts for each expert (16 experts for DBRX)
        counts = torch.zeros(16)
        for expert_id, count in layer_data.items():
            if 0 <= expert_id < 16:  # Ensure expert ID is valid
                counts[expert_id] = count
        expert_token_counts[layer_num] = counts
    
    # Save just the token counts per expert
    torch.save(expert_token_counts, output_path)
    print(f"Expert token counts saved to {output_path}")
    
    # Create a DataFrame for visualization purposes
    rows = []
    for layer_num in sorted(expert_counts.keys()):
        layer_data = expert_counts[layer_num]
        for expert_id in range(16):  # 16 experts for DBRX
            count = layer_data.get(expert_id, 0)
            rows.append({
                'layer': layer_num,
                'expert_id': expert_id,
                'token_count': count
            })
    
    df = pd.DataFrame(rows)
    return df

def analyze_text_file_routing(model, tokenizer, file_path):
    """
    Main function to analyze MoE routing for a text file.
    
    Args:
        file_path: Path to text file with prompts (one per line)
        model_name: Name of DeepSeek MoE model to use
    """
    # Process the file for expert counts and save as PyTorch file
    df = process_text_file_for_expert_counts(file_path, model, tokenizer)
    
    print(f"Analysis completed for {file_path}")
    return df

In [ ]:
file_path = "data-ext/test.txt"
df = analyze_text_file_routing(model, tokenizer, file_path)


In [ ]:
data = torch.load("data-ext/gsm8k_expert_data.pt")

In [ ]:
data[1]

In [46]:
def bar_graph_all_tokens_paper(expert_data, layer_number, tokenizer, domain=None):
    """
    Visualizes expert distribution for all tokens in a file for a specific layer.
    Adapted for DBRX model which has 16 experts rather than 64.
    
    Args:
        expert_data: Dictionary with layer numbers as keys and tensor of expert counts as values
                     or path to PyTorch file with this data
        layer_number: Layer to analyze
        tokenizer: DBRX tokenizer
        domain: Optional domain name for title (e.g., 'GSM8K', 'Math', etc.)
    
    Returns:
        fig: Plotly figure object
    """
    # Load data if a file path is provided
    if isinstance(expert_data, str):
        txt_file_path = expert_data.replace("data-ext/pt/", "data-ext/").replace("_expert_data.pt", ".txt").replace("_expert_data_chat.pt", ".txt").replace("_expert_data_base.pt", ".txt")
        
        num_tokens = None
        if os.path.exists(txt_file_path):
            # Try different encodings
            encodings = ['utf-8', 'latin-1', 'utf-16']
            for encoding in encodings:
                try:
                    with open(txt_file_path, 'r', encoding=encoding) as f:
                        text_content = f.read()
                        tokens = tokenizer(text_content, return_tensors="pt")
                        num_tokens = tokens.input_ids.numel()
                        print(f"Total tokens in {txt_file_path}: {num_tokens}")
                        break
                except UnicodeDecodeError:
                    continue
            else:
                print(f"Could not read {txt_file_path} with any of the attempted encodings")
        
        expert_data = torch.load(expert_data)
    else:
        # If expert_data is already loaded (not a string path)
        num_tokens = None
    
    # Validate layer number is in the data
    if layer_number not in expert_data:
        raise ValueError(f"Layer {layer_number} not found in expert data")
    
    # Get counts for the specified layer
    expert_counts = expert_data[layer_number].numpy()
    
    # For DBRX, we expect 16 experts rather than 64
    num_experts = len(expert_counts)
    if num_experts != 16:
        print(f"Warning: Expected 16 experts for DBRX, but found {num_experts}")
        
    # Compute percentages
    total_tokens = num_tokens or expert_counts.sum() / 4  # Each token routes to 4 experts in DBRX
    if total_tokens == 0:
        print("No tokens found for this layer")
        return
    
    print(f"total_tokens: {total_tokens}")
    print(f"num_tokens: {num_tokens}")
    percentages = (expert_counts / total_tokens) * 100
    
    # Set discrete opacity based on 25% threshold (4 times expected uniform distribution of 1/16 = 6.25%)
    threshold = 25.0
    opacities = np.where(percentages >= threshold, 1.0, 0.3)
    
    # Create plotly figure
    fig = go.Figure()
    
    # Add bar trace with discrete color and opacity
    fig.add_trace(go.Bar(
        x=list(range(num_experts)),
        y=percentages,
        marker=dict(
            color='#636EFA',  # Blue color for all bars
            opacity=opacities
        ),
        hovertemplate='Expert ID: %{x}<br>Tokens: %{text}<br>Percentage: %{y:.2f}%<extra></extra>',
        text=[f"{int(count)}" for count in expert_counts],
        textposition='none'  # Ensure no text is displayed on the bars
    ))
    
    # Add horizontal line at threshold
    fig.add_shape(
        type="line",
        x0=-0.5,
        x1=num_experts-0.5,
        y0=threshold,
        y1=threshold,
        line=dict(
            color="red",
            width=2,
            dash="dash",
        )
    )
    
    # Add annotation for the threshold line
    fig.add_annotation(
        x=num_experts-1,
        y=threshold,
        text=f"{threshold}% threshold",
        showarrow=False,
        yshift=10,
        font=dict(color="red")
    )
    
    routed_tokens = expert_counts.sum()
    
    # Domain label for title
    domain_label = f" - {domain}" if domain else ""
    token_info = f" (No. of Tokens: {num_tokens}, Routed Tokens: {int(routed_tokens)})" if num_tokens else f" (Routed Tokens: {int(routed_tokens)})"
    
    # Update layout with white background
    fig.update_layout(
        title=f'Expert Usage Distribution - Layer {layer_number}{domain_label}{token_info}',
        xaxis_title='Expert ID',
        yaxis_title='Usage Percentage (%)',
        yaxis_range=[0, 100],
        xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Set tick for each expert (0-15)
        showlegend=False,
        width=1000,
        height=600,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # Add gridlines with lighter color
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128, 128, 128, 0.1)')
    fig.update_xaxes(showgrid=False)
    
    return fig

In [ ]:
fig = bar_graph_all_tokens_paper("data-ext/arxiv_title_abstract_expert_data_base.pt", layer_number=15, tokenizer=tokenizer)
fig.show()